In [1]:
import requests
import psycopg2
import polyline
import gpxpy
import json

import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
OUTPUT_DIR = '../routes'

In [3]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
def get_route(points):
    params = {
        'geometries': 'polyline6',
    }
    response = requests.get(f"http://127.0.0.1:6000/route/v1/foot/{points}", params=params)
    print(response.json())
    response.raise_for_status()
    route = response.json()
    return route['routes'][0]['geometry']

In [ ]:
def get_nearest(point):
    response = requests.get(f"http://127.0.0.1:6000/nearest/v1/foot/{point}")
    print(response.json())
    response.raise_for_status()
    waypoints = response.json()
    location = waypoints['waypoints'][0]['location']
    return f"{location[0]},{location[1]}"

In [ ]:
def plot_gpx(gpx_df, filename):
    plt.figure(figsize=(15, 8))
    plt.plot(gpx_df['longitude'], gpx_df['latitude'], color='red', label='Route')
    ctx.add_basemap(plt.gca(), crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
    plt.legend()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    plt.savefig(f"{OUTPUT_DIR}/{filename}.png")

In [4]:
query = """
SELECT
    b.name AS destination,
    a.latitude AS start_lat,
    a.longitude AS start_lon, 
    b.latitude AS stop_lat,
    b.longitude AS stop_lon
FROM start a
CROSS JOIN destinations b
"""

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()

df = pd.DataFrame(rows, columns=[
    "destination",
    "start_lat", 
    "start_lon", 
    "stop_lat",
    "stop_lon",
])

df.head(3)

NameError: name 'conn' is not defined

In [5]:
for index, row in df.iterrows():    
    start_location = get_nearest(f"{row['start_lon']},{row['start_lat']}")
    end_location = get_nearest(f"{row['stop_lon']},{row['stop_lat']}")
    
    points=f"{start_location};{end_location}"
    print(points)
    
    try:
        geometry = get_route(points)
#         coordinates = polyline.decode(geometry, 6)

#         gpx_df = pd.DataFrame(coordinates, columns=['latitude', 'longitude'])
#         plot_gpx(gpx_df, row['destination'])

#         distance = round(row['distance_to_destination'] / 1000.0)
#         filename = f"{row['destination']} ({distance}km)"
#         create_gpx(coordinates, filename)
    except:
        continue

NameError: name 'df' is not defined